# Notebook to remove confounds from a cifti image and apply LPF

There seem to be 2 functions for removing confounds, filtering, etc:

- https://nilearn.github.io/modules/generated/nilearn.image.clean_img.html
- https://nilearn.github.io/modules/generated/nilearn.signal.clean.html

Here I will apply clean_img to the cifti data

In [2]:
import numpy as np
from scipy import signal
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set("paper", "white")

from nilearn.image import load_img
from nilearn.image import clean_img
from nilearn.image import smooth_img
from nilearn.image import index_img
from nilearn.signal import clean
from os import walk

from utility import var_to_nan


import nibabel as nib
from nibabel import cifti2
%matplotlib inline

## Load a sub whole brain data

In [3]:
brainpath='/om2/user/jsmentch/projects/nat_img/sourcedata/data/budapest/brain/sub-sid000007_task-movie_run-1_space-fsLR_den-91k_bold.dtseries.nii'
brainimg = load_img(brainpath) # load func img

In [8]:
confound_path = '/om2/user/jsmentch/projects/nat_img/sourcedata/data/budapest/brain/sub-sid000007_task-movie_run-1_desc-confounds_timeseries.tsv'

In [9]:
data = pd.read_csv(confound_path, sep='\t')
confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()

In [12]:
func_data=np.array(brainimg.dataobj)
func_data = var_to_nan(func_data,0.5)

In [14]:
#func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=1)
func_cln = nib.Cifti2Image(func_data_clean, brainimg.header)

In [ ]:
func_cln.to_filename(export_path+sub+'_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii')

In [2]:
# # fn to load a subject func scan
# def load_sub_data(sub):
#     directory_string = "/om2/user/jsmentch/data/datalad/ds001110/derivatives/fmriprep_old/" + sub + "/func/"
#     nifti_file = "_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii"
#     #load subject's functional data
#     nii_file = directory_string + sub + nifti_file
#     brainimg = load_img(nii_file) # load func img
#     return brainimg

In [3]:
# # list of all subjects
# #subject_list = list(walk("/om2/user/jsmentch/neuroscout/out/wl8RX/fitlins"))[0][1][:-1] #exclude last sub-36
# subject_list = list(walk("/om2/user/jsmentch/neuroscout/out/wl8RX/fitlins"))[0][1]
# subject_list = subject_list[1:]

In [6]:
#subject_list.remove('sub-21') # exclude subject 21 because their data is not there?

In [ ]:
# for sub in subject_list:
#     func = load_sub_data(sub)
#     print(func.shape)
# #sub = subject_list[1] #first sub

## Test it out on one person

In [6]:
# sub = 'sub-19'
# func = load_sub_data(sub)
# func.shape

(1030, 91282)

### Load confounds

In [7]:
# confound_path = "/om2/user/jsmentch/data/datalad/ds001110/derivatives/fmriprep_old/" + sub + "/func/" + sub + "_task-MerlinMovie_desc-confounds_regressors.tsv"
# data = pd.read_csv(confound_path, sep='\t')
# confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()

clean_img: detrend, remove confounds, low pass at 0.1 Hz

### nilearn doesn't support cifti for clean_img so use clean()

In [11]:
func_data=np.array(func.dataobj)
func_data = var_to_nan(func_data,0.5)

In [12]:
#func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=1.5)
func_cln = nib.Cifti2Image(func_data_clean, func.header)

In [48]:
print(func_cln)

## Now do this for everyone

In [15]:
def clean_sub(sub):
    #load confounds for each subject
    confound_path = "/om2/user/jsmentch/data/datalad/ds001110/derivatives/fmriprep_old/" + sub + "/func/" + sub + "_task-MerlinMovie_desc-confounds_regressors.tsv"
    data = pd.read_csv(confound_path, sep='\t')
    confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
    func = load_sub_data(sub) #load cifti
    #clean
    func_data=np.array(func.dataobj)
    func_data = var_to_nan(func_data,0.5)
    #func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
    func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=1.5)
    func_cln = nib.Cifti2Image(func_data_clean, func.header)
    #smooth 2mm
    #func_cln_smth = smooth_img(func_cln,2)
    return func_cln

In [16]:
export_path = '/om2/user/jsmentch/projects/speech_face_analysis/data/cifti/cleaned/'
for sub in subject_list:
    func_cln = clean_sub(sub)
    print(f'exporting {sub}')
    func_cln.to_filename(export_path+sub+'_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii')

exporting sub-19
exporting sub-20
exporting sub-21
exporting sub-22
exporting sub-23
exporting sub-24
exporting sub-25
exporting sub-26
exporting sub-27
exporting sub-28
exporting sub-29
exporting sub-30
exporting sub-31
exporting sub-32
exporting sub-33
exporting sub-34
exporting sub-35
exporting sub-36


In [ ]:
#ignore below here for ciftis

In [8]:
cleanpath='../data/cifti/cleaned/smoothed/'
subject_flist = list(walk(cleanpath))[0][2:][0]
subject_flist

['sub-19_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-20_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-21_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-22_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-23_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-24_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-25_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-26_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-27_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-28_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-29_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-30_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii',
 'sub-31_clean_smooth_task-MerlinMovie_space-fsLR_den-91k_bold.d

In [9]:
s_f=subject_flist[0]
img = nib.load(cleanpath+str(s_f))
all_sub_added = img.get_fdata()[0,:]
for s_f in subject_flist:
    img = nib.load(cleanpath+str(s_f))
    X = img.get_fdata()[0,:] #load data from nii
    all_sub_added = all_sub_added+X

union_ind = np.where(np.isnan(all_sub_added),float('NaN'),1) #array where union is 1 and other is NaN so if you * it to another array it will make things NaN that are not in the union

In [10]:
#import data
cleanpath='../data/cifti/cleaned/'
subject_flist = list(walk(cleanpath))[0][2:][0]
braintrain = []
braintest = []
for s_f in subject_flist:
    print(s_f)
    img = nib.load(cleanpath+str(s_f))
    X = img.get_fdata()*union_ind #load data from nii, set things outside group to NaN
    X_t = X[17:] #trim beginning, first 17 TRs
    s_brain = X_t[:1009] #trim end to end of film    braintrain.append(s_brain[:-200,:]) #roughly 80 20 split, trim the last 200 TRs of each subject to save as test set
    braintrain.append(s_brain[:-200,:])
    braintest.append(s_brain[-200:,:])
Ybrain_train = np.vstack(braintrain)
Ybrain_test = np.vstack(braintest)


sub-19_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-20_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-21_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-22_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-23_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-24_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-25_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-26_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-27_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-28_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-29_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-30_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-31_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-32_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii
sub-33_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries

In [11]:
Ybrain_train = np.nan_to_num(Ybrain_train)
Ybrain_test = np.nan_to_num(Ybrain_test)

In [12]:
np.save('data/Ybrain_train.npy',Ybrain_train)
np.save('data/Ybrain_test.npy',Ybrain_test)

In [56]:
func.dataobj[0,...].shape

(1030, 91282)

In [8]:
# fn to load a subjects cleaned func scan
def load_decon_data(sub):
    directory_string = '/om2/user/jsmentch/data/brain/merlin/func_clean/' 
    nifti_file = sub + "_clean.nii.gz"
    #load subject's functional data
    nii_file = directory_string + nifti_file
    brainimg = load_img(nii_file) # load func img
    return brainimg

In [22]:
def trim_sub(sub):
    func = load_decon_data(sub)
    #clean - double check, are these confounds in the correct format?
    func_trim = index_img(func,slice(17, 1026)) #remove the first 17 TRs and the shortest is len 1026 so trim the ends of the others
    return func_trim

In [24]:
index_img(func,slice(17, 1026))

TypeError: Data given cannot be loaded because it is not compatible with nibabel format:
Cifti2Image('/om2/...

In [10]:
export_path = '/om2/user/jsmentch/data/brain/merlin/func_clean_trimmed/'
for sub in subject_list:
    func_trim = trim_sub(sub)
    print(f'exporting {sub}')
    func_trim.to_filename(export_path+sub+'_trimmed.nii.gz')    

exporting sub-19
exporting sub-20
exporting sub-22
exporting sub-23
exporting sub-24
exporting sub-25
exporting sub-26
exporting sub-27
exporting sub-28
exporting sub-29
exporting sub-30
exporting sub-31
exporting sub-32
exporting sub-33
exporting sub-34
exporting sub-35


In [22]:
# fn to apply a mask to a nii file
def apply_roi_mask(data, mask, template):
    #load masks of auditory region
    roi_dir = "/om2/user/jsmentch/data/rois/svnh_ROIS_anatlabels_surf_mni/mni15\
2_te11-te10-te12-pt-pp/"
    roi_mask = load_img(roi_dir + mask)
    #Mask
    data_masked = apply_mask(data, roi_mask) #apply brain mask to data
    return data_masked

In [23]:
template = load_mni152_template() # load mni152 template

subject_list = list(walk("/om2/user/jsmentch/neuroscout/out/wl8RX/fitlins"))[0][1][:-1]
roi_list = listdir("/om2/user/jsmentch/data/rois/svnh_ROIS_anatlabels_surf_mni/mni152_te11-te10-te12-pt-pp")[:-3] #exclude lh, rh, lh_rh

NameError: name 'load_mni152_template' is not defined

In [ ]:
for sub in subject_list:
    data = load_sub_data(sub)
    for mask in roi_list:
        data_masked = apply_roi_mask(data, mask, template)
        out_dir = "/om2/user/jsmentch/data/" + sub + "/"
        Path(out_dir).mkdir(parents=True, exist_ok=True)
        save_name = out_dir + sub + "_" + mask.split('.')[0] + "_" + mask.split\
('.')[1]
        np.save(save_name, data_masked)

In [ ]:
# script to process a list of the merlin subjects and masks, apply them all
from nilearn.image import load_img
from nilearn.datasets import load_mni152_template
from nilearn.image import resample_to_img
from nilearn.masking import apply_mask
import numpy as np
from pathlib import Path
from os import listdir
